In [ ]:
import pandas as pd
import glob
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path for An
opath = '/content/drive/MyDrive/01. Master Degree/03. Data Mining/Group 8 Project'

In [ ]:
# Path for Owen
opath = '/content/drive/MyDrive/umn/24-25/zfall 25/csci 5523/Group 8 Project'

In [ ]:
# Path for Ahmed
# opath =

In [ ]:
# Path for Nour
# opath =

# 1. Call for merged data

In [ ]:
preprocessed_all = os.path.join(opath, 'Merged Dataframe')

In [ ]:
calming_df = pd.read_csv(os.path.join(preprocessed_all, 'Calming_Merged_feature.csv'))
vexing_df = pd.read_csv(os.path.join(preprocessed_all, 'Vexing_Merged_feature.csv'))
# music_df = pd.read_csv(os.path.join(preprocessed_all, 'Music_df.csv'))

In [ ]:
calming_df.head()

,Unnamed: 0,n_back,Response_Time,condition,subject_id,BlockNumber,TrialInBlock,Correct,BVP_mean,BVP_std,...,ECG_mean,ECG_std,EMG_mean,EMG_std,RESP_mean,RESP_std,PPG_mean,PPG_std,SKT_mean,SKT_std
0,0,1,0.0,Calming,Subject4,1,1,0,-2.751331,5.453607,...,0.010924,0.195181,-0.002213,0.017522,-6.161784,0.105731,0.050346,0.024017,22.968318,0.000834
1,1,1,838.0,Calming,Subject4,1,2,1,4.456672,0.879887,...,0.010689,0.192964,-0.002241,0.017178,-6.033325,0.126568,0.050484,0.019253,22.968283,0.000822
2,2,1,374.0,Calming,Subject4,1,3,1,4.285210,0.740332,...,0.017447,0.221070,-0.002368,0.018552,-6.090992,0.086868,0.050917,0.020832,22.968333,0.000821
3,3,1,405.0,Calming,Subject4,1,4,1,-2.634205,2.918769,...,0.000513,0.216476,-0.002110,0.020708,-5.981885,0.137434,0.046472,0.021422,22.968352,0.000817
4,4,1,39.0,Calming,Subject4,1,5,1,0.772251,0.211038,...,0.006566,0.199935,-0.002105,0.022657,-6.062608,0.227627,0.048821,0.016176,22.968353,0.000816


# 2. Data splitting

In [ ]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

def split_data(df, model_predicting = 'n_back'):
  '''
  In this function, we will split the data into training dataset, validation dataset, and testing dataset
  Args:
  df: Dataframe of raw data
  model_predicting: [str] indicating which model we are trying to split for
  '''

  if model_predicting == 'n_back':
    df_filtered = df[df['n_back'] != 'rest'].copy()
    y = df_filtered['n_back'].astype(int)
    columns_to_drop = [
        'n_back', 'subject_id', 'BlockNumber', 'TrialInBlock',
        'Session', 'Block', 'Trial', 'task' # These are from empatica_df, might not exist in merged df
    ]
    # Ensure only existing columns are dropped
    final_columns_to_drop = [col for col in columns_to_drop if col in df_filtered.columns]
    X = df_filtered.drop(columns=final_columns_to_drop)
    # One-hot encode 'condition' columns
    X = pd.get_dummies(X, columns=['condition'], drop_first=True)

  elif model_predicting == 'condition':
    df_filtered = df[df['n_back'] != 'rest'].copy()
    y = df_filtered['condition'].map({'calming': 0, 'vexing': 1})
    columns_to_drop = ['BlockNumber', 'TrialInBlock', 'subject_id', 'condition', 'task', 'n_back']

    final_columns_to_drop = [col for col in columns_to_drop if col in df_filtered.columns]
    X = df_filtered.drop(columns=final_columns_to_drop)

  groups = df_filtered['subject_id']

  # Split 1: Create the Test set (10% of subjects)
  gss_test = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
  train_val_idx, test_idx = next(gss_test.split(X, y, groups))

  X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
  y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]
  groups_train_val = groups.iloc[train_val_idx] # Get the subject list for the 90%

  # Split 2: Create the Validation set (10% of total)
  # We take 10% from the remaining 90% (0.10 / 0.90 = ~0.111)
  val_split_size = 0.10 / 0.90
  gss_valid = GroupShuffleSplit(n_splits=1, test_size=val_split_size, random_state=42)
  train_idx, valid_idx = next(gss_valid.split(X_train_val, y_train_val, groups_train_val))

  # Create the final sets
  X_train, X_valid = X_train_val.iloc[train_idx], X_train_val.iloc[valid_idx]
  y_train, y_valid = y_train_val.iloc[train_idx], y_train_val.iloc[valid_idx]

  print("\n Split Verification (80/10/10)")
  print(f"Total rows in X_train: {len(X_train)} (~80%)")
  print(f"Total rows in X_valid: {len(X_valid)} (~10%)")
  print(f"Total rows in X_test:  {len(X_test)} (~10%)")

  print(f"\nTraining subjects:   {groups_train_val.iloc[train_idx].unique()}")
  print(f"Validation subjects: {groups_train_val.iloc[valid_idx].unique()}")
  print(f"Test subjects:       {groups.iloc[test_idx].unique()}")

  # Check for any overlap (should be empty sets)
  train_subs = set(groups_train_val.iloc[train_idx].unique())
  valid_subs = set(groups_train_val.iloc[valid_idx].unique())
  test_subs = set(groups.iloc[test_idx].unique())

  print(f"\nOverlap Train/Valid: {train_subs.intersection(valid_subs)}")
  print(f"Overlap Train/Test:  {train_subs.intersection(test_subs)}")
  print(f"Overlap Valid/Test:  {valid_subs.intersection(test_subs)}")

  return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
calm_X_train, calm_X_valid, calm_X_test, calm_y_train, calm_y_valid, calm_y_test = split_data(calming_df)
vex_X_train, vex_X_valid, vex_X_test, vex_y_train, vex_y_valid, vex_y_test = split_data(vexing_df)


 Split Verification (80/10/10)
Total rows in X_train: 1056 (~80%)
Total rows in X_valid: 352 (~10%)
Total rows in X_test:  352 (~10%)

Training subjects:   ['Subject6' 'Subject11' 'Subject8']
Validation subjects: ['Subject4']
Test subjects:       ['Subject3']

Overlap Train/Valid: set()
Overlap Train/Test:  set()
Overlap Valid/Test:  set()

 Split Verification (80/10/10)
Total rows in X_train: 1056 (~80%)
Total rows in X_valid: 352 (~10%)
Total rows in X_test:  352 (~10%)

Training subjects:   ['Subject6' 'Subject8' 'Subject11']
Validation subjects: ['Subject4']
Test subjects:       ['Subject3']

Overlap Train/Valid: set()
Overlap Train/Test:  set()
Overlap Valid/Test:  set()


In [ ]:
calm_y_train.head()

,n_back
352,1
353,1
354,1
355,1
356,1


# 3. Feature selection

In [ ]:
def feature_engineering(X_df, y_df, drop_threshold):
  '''
  This function returns a list of features that have correlation > threshold
  Args:
  X_df: X training dataframe
  y_df: y training dataframe

  Returns:
  X_df_feature_engineering: list of index
  '''
  # Computing the correlation of each feature against the label 'n-back'
  X_df_feature_engineering = X_df.corrwith(y_df)

  print("\nFeature correlations:")
  print(abs(X_df_feature_engineering).sort_values(ascending=False))

  # Dropping all features that have correlation < threshold
  X_df_feature_engineering = X_df_feature_engineering[abs(X_df_feature_engineering) > drop_threshold]

  print("\nSelecting features that have correlation > {} with n-back label".format(drop_threshold))
  print(X_df_feature_engineering)
  print(f"\n{'='*40}")
  return X_df_feature_engineering.index.to_list()

In [ ]:
calm_X_train_FE = feature_engineering(calm_X_train, calm_y_train, 0.05)
vex_X_train_FE = feature_engineering(vex_X_train, vex_y_train, 0.03)


Feature correlations:
Correct              0.246588
HR_std               0.124619
Response_Time        0.107886
RESP_mean            0.059623
RESP_std             0.047054
SKT_std              0.039030
ECG_std              0.038174
EDA_std_empatica     0.032715
ACC_std              0.032237
EDA_std_biopac       0.029649
PPG_mean             0.028711
ECG_mean             0.026279
EMG_std              0.023868
BVP_std              0.022212
IBI_std              0.018356
TEMP_mean            0.018070
PPG_std              0.017382
SKT_mean             0.015683
IBI_mean             0.015456
EDA_mean_biopac      0.014946
HR_mean              0.011025
EMG_mean             0.009455
Unnamed: 0           0.009021
EDA_mean_empatica    0.006749
BVP_mean             0.004331
ACC_mean             0.003182
dtype: float64

Selecting features that have correlation > 0.05 with n-back label
Response_Time    0.107886
Correct         -0.246588
HR_std          -0.124619
RESP_mean        0.059623
dtype: floa

In [ ]:
calm_X_retrain = calm_X_train[calm_X_train_FE]
vex_X_retrain = vex_X_train[vex_X_train_FE]

In [ ]:
calm_X_revalid = calm_X_valid[calm_X_train_FE]
vex_X_revalid = vex_X_valid[vex_X_train_FE]

calm_X_retest = calm_X_test[calm_X_train_FE]
vex_X_retest = vex_X_test[vex_X_train_FE]

# 4. Deep Neural Network

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import numpy as np

# 1. Define Custom Dataset
class NBackDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        # For binary classification with BCEWithLogitsLoss, target y should be float tensor of shape (N, 1)
        self.y = torch.FloatTensor(y).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 2. Define Neural Network for Binary Classification
class BinaryClassifierNN(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifierNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

        self.layer2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)

        self.output = nn.Linear(32, 1) # Single output for binary classification

    def forward(self, x):
        x = self.relu(self.bn1(self.layer1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.layer2(x)))
        x = self.output(x)
        return x

# 3. Helper Function to Calculate Metrics
def get_metrics(model, loader, device):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            # Apply sigmoid and round for binary predictions
            preds = torch.round(torch.sigmoid(outputs))

            all_preds.extend(preds.flatten().cpu().numpy())
            all_targets.extend(targets.flatten().cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)
    # Use 'binary' average for binary classification metrics, specifying pos_label=1
    prec = precision_score(all_targets, all_preds, average='binary', pos_label=1, zero_division=0)
    rec = recall_score(all_targets, all_preds, average='binary', pos_label=1, zero_division=0)
    f1 = f1_score(all_targets, all_preds, average='binary', pos_label=1, zero_division=0)
    return acc, prec, rec, f1

def train_and_evaluate(X_train_df, y_train_s, X_val_df, y_val_s,
                       batch_size=32, learning_rate=0.001, epochs=20):

    # A. Map binary labels (1 and 3) to 0 and 1
    y_train_arr_original = y_train_s.values if hasattr(y_train_s, 'values') else y_train_s
    y_val_arr_original = y_val_s.values if hasattr(y_val_s, 'values') else y_val_s

    # Assuming 1 is the negative class (0) and 3 is the positive class (1)
    print("INFO: Mapping target labels: 1 -> 0, 3 -> 1 for binary classification.")
    y_train_arr = (y_train_arr_original == 3).astype(float)
    y_val_arr = (y_val_arr_original == 3).astype(float)

    # B. Scaling (CRITICAL for Neural Networks)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_df)
    X_val_scaled = scaler.transform(X_val_df)

    # C. Create Datasets and Loaders
    train_dataset = NBackDataset(X_train_scaled, y_train_arr)
    val_dataset = NBackDataset(X_val_scaled, y_val_arr)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    # D. Setup Model, Loss, Optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_dim = X_train_scaled.shape[1]
    model = BinaryClassifierNN(input_dim).to(device) # Use BinaryClassifierNN

    criterion = nn.BCEWithLogitsLoss() # Changed back to BCEWithLogitsLoss for binary
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"Training on {device} with {input_dim} features for binary classification.")
    print(f"{'Epoch':<6} | {'Train Loss':<10} | {'Val Acc':<8} | {'Val Prec':<8} | {'Val Rec':<8} | {'Val F1':<8}")
    print("-" * 65)

    # E. Training Loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        epoch_loss = running_loss / len(train_loader)

        # Calculate Metrics
        train_acc, train_prec, train_rec, train_f1 = get_metrics(model, train_eval_loader, device)
        val_acc, val_prec, val_rec, val_f1 = get_metrics(model, val_loader, device)

        print(f"{epoch+1:<6} | {epoch_loss:<10.4f} | {val_acc:<8.4f} | {val_prec:<8.4f} | {val_rec:<8.4f} | {val_f1:<8.4f}")

    return model, scaler

## Deep Neural Network with no feature selection

In [33]:
from sklearn.preprocessing import StandardScaler

# Initialize scalers for each condition
calm_scaler_X = StandardScaler()
vex_scaler_X = StandardScaler()

# Fit and transform training data for calming condition
calm_X_train_scaled = calm_scaler_X.fit_transform(calm_X_train)
calm_y_train_np = calm_y_train.values

# Transform validation and test data for calming condition
calm_X_valid_scaled = calm_scaler_X.transform(calm_X_valid)
calm_y_valid_np = calm_y_valid.values

calm_X_test_scaled = calm_scaler_X.transform(calm_X_test)
calm_y_test_np = calm_y_test.values

# Fit and transform training data for vexing condition
vex_X_train_scaled = vex_scaler_X.fit_transform(vex_X_train)
vex_y_train_np = vex_y_train.values

# Transform validation and test data for vexing condition
vex_X_valid_scaled = vex_scaler_X.transform(vex_X_valid)
vex_y_valid_np = vex_y_valid.values

vex_X_test_scaled = vex_scaler_X.transform(vex_X_test)
vex_y_test_np = vex_y_test.values

# Build tensor datasets using the scaled numpy arrays
calm_train_dataset = NBackDataset(calm_X_train_scaled, calm_y_train_np)
vex_train_dataset = NBackDataset(vex_X_train_scaled, vex_y_train_np)

calm_validation_dataset = NBackDataset(calm_X_valid_scaled, calm_y_valid_np)
vex_validation_dataset = NBackDataset(vex_X_valid_scaled, vex_y_valid_np)

calm_test_dataset = NBackDataset(calm_X_test_scaled, calm_y_test_np)
vex_test_dataset = NBackDataset(vex_X_test_scaled, vex_y_test_np)

In [ ]:
# calm_X_train, calm_X_valid, calm_X_test, calm_y_train, calm_y_valid, calm_y_test = split_data(calming_df)
# vex_X_train, vex_X_valid, vex_X_test, vex_y_train, vex_y_valid, vex_y_test = split_data(vexing_df)

In [34]:
# Train and evaluate the neural network model for calming condition
calm_model, calm_scaler = train_and_evaluate(
    calm_X_train, calm_y_train,
    calm_X_valid, calm_y_valid,
    epochs=150
)

INFO: Mapping target labels: 1 -> 0, 3 -> 1 for binary classification.
Training on cpu with 26 features for binary classification.
Epoch  | Train Loss | Val Acc  | Val Prec | Val Rec  | Val F1  
-----------------------------------------------------------------


1      | 0.6872     | 0.5455   | 0.5261   | 0.9148   | 0.6680  


2      | 0.6593     | 0.5710   | 0.5451   | 0.8580   | 0.6667  


3      | 0.6291     | 0.6023   | 0.5882   | 0.6818   | 0.6316  


4      | 0.6121     | 0.6165   | 0.6340   | 0.5511   | 0.5897  


...

146    | 0.2497     | 0.4716   | 0.4038   | 0.1193   | 0.1842  


147    | 0.2505     | 0.4915   | 0.4892   | 0.3864   | 0.4317  


148    | 0.2608     | 0.4915   | 0.4862   | 0.3011   | 0.3719  


149    | 0.2674     | 0.4972   | 0.4963   | 0.3807   | 0.4309  


150    | 0.2467     | 0.5057   | 0.5059   | 0.4886   | 0.4971  


In [ ]:
# Train and evaluate the neural network model for calming condition
vex_model, vex_scaler = train_and_evaluate(
    vex_X_train, vex_y_train,
    vex_X_valid, vex_y_valid,
    epochs=150
)

INFO: Mapping target labels: 1 -> 0, 3 -> 1 for binary classification.
Training on cpu with 26 features for binary classification.
Epoch  | Train Loss | Val Acc  | Val Prec | Val Rec  | Val F1  
-----------------------------------------------------------------


1      | 0.6794     | 0.5455   | 0.6905   | 0.1648   | 0.2661  


2      | 0.6468     | 0.5966   | 0.6164   | 0.5114   | 0.5590  


3      | 0.6127     | 0.5568   | 0.5581   | 0.5455   | 0.5517  


4      | 0.5976     | 0.5938   | 0.6241   | 0.4716   | 0.5372  


...

84     | 0.3518     | 0.4773   | 0.4868   | 0.8409   | 0.6167  


85     | 0.3427     | 0.4716   | 0.4840   | 0.8580   | 0.6189  


86     | 0.3522     | 0.4716   | 0.4829   | 0.8011   | 0.6026  


87     | 0.3407     | 0.4801   | 0.4879   | 0.8011   | 0.6065  


Epoch 88/150:   0%|          | 0/33 [00:00<?, ?it/s, loss=0.2238]

## Deep Neural Network with feature optimization

In [35]:
# Train and evaluate the neural network model for calming condition
calm_model, calm_scaler = train_and_evaluate(
    calm_X_retrain, calm_y_train,
    calm_X_revalid, calm_y_valid,
    epochs=100
)

INFO: Mapping target labels: 1 -> 0, 3 -> 1 for binary classification.
Training on cpu with 4 features for binary classification.
Epoch  | Train Loss | Val Acc  | Val Prec | Val Rec  | Val F1  
-----------------------------------------------------------------


1      | 0.6692     | 0.6278   | 0.6364   | 0.5966   | 0.6158  


2      | 0.6449     | 0.6705   | 0.7143   | 0.5682   | 0.6329  


3      | 0.6371     | 0.6733   | 0.7163   | 0.5739   | 0.6372  


4      | 0.6303     | 0.6761   | 0.7313   | 0.5568   | 0.6323  


...

96     | 0.6029     | 0.6733   | 0.6943   | 0.6193   | 0.6547  


97     | 0.5909     | 0.6648   | 0.7014   | 0.5739   | 0.6312  


98     | 0.6072     | 0.6676   | 0.6954   | 0.5966   | 0.6422  


99     | 0.6024     | 0.6676   | 0.7063   | 0.5739   | 0.6332  


100    | 0.5903     | 0.6676   | 0.7122   | 0.5625   | 0.6286  


In [37]:
# Train and evaluate the neural network model for calming condition
vex_model, vex_scaler = train_and_evaluate(
    vex_X_retrain, vex_y_train,
    vex_X_revalid, vex_y_valid,
    epochs=100
)

INFO: Mapping target labels: 1 -> 0, 3 -> 1 for binary classification.
Training on cpu with 8 features for binary classification.
Epoch  | Train Loss | Val Acc  | Val Prec | Val Rec  | Val F1  
-----------------------------------------------------------------


1      | 0.6714     | 0.5568   | 0.5781   | 0.4205   | 0.4868  


2      | 0.6295     | 0.5795   | 0.8333   | 0.1989   | 0.3211  


3      | 0.6165     | 0.6023   | 0.8600   | 0.2443   | 0.3805  


4      | 0.6107     | 0.5824   | 0.8372   | 0.2045   | 0.3288  


...

96     | 0.5576     | 0.6250   | 0.7973   | 0.3352   | 0.4720  


97     | 0.5606     | 0.6080   | 0.8654   | 0.2557   | 0.3947  


98     | 0.5553     | 0.6193   | 0.8750   | 0.2784   | 0.4224  


99     | 0.5563     | 0.6222   | 0.8209   | 0.3125   | 0.4527  


100    | 0.5594     | 0.6023   | 0.8600   | 0.2443   | 0.3805  
